In [194]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


In [195]:
df = pd.read_csv('/content/covid_toy.csv')

In [196]:
df


,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [198]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [199]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2,random_state=42)

In [200]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((80, 5), (20, 5), (80,), (20,))

In [201]:
x_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi
...,...,...,...,...,...
60,24,Female,102.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi
14,51,Male,104.0,Mild,Bangalore
92,82,Female,102.0,Strong,Kolkata


In [202]:
## WITHOUT PIPELINES

In [203]:
si  = SimpleImputer()

si.fit(x_train[['fever']])

x_train_fever = si.transform(x_train[['fever']])
x_test_fever = si.transform(x_test[['fever']])


In [204]:
x_train_fever.shape,x_test_fever.shape

((80, 1), (20, 1))

In [205]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])

oe.fit(x_train[['cough']])

x_train_cough = oe.transform(x_train[['cough']])
x_test_cough = oe.transform(x_test[['cough']])


In [206]:
x_train_cough.shape,x_test_cough.shape

((80, 1), (20, 1))

In [207]:
ohe = OneHotEncoder(drop='first',sparse_output=False)

ohe.fit(x_train[['gender','city']])

x_train_gender_city = ohe.transform(x_train[['gender','city']])
x_test_gender_city = ohe.transform(x_test[['gender','city']])

In [208]:
x_train_gender_city.shape,x_test_gender_city.shape

((80, 4), (20, 4))

In [209]:
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

In [210]:
x_train_age.shape,x_test_age.shape

((80, 1), (20, 1))

In [211]:
x_train_new = np.concatenate((x_train_age,x_train_cough,x_train_fever,x_train_gender_city),axis=1)
x_test_new = np.concatenate((x_test_age,x_test_cough,x_test_fever,x_test_gender_city),axis=1)

In [212]:
x_train_new.shape,x_test_new.shape

((80, 7), (20, 7))

In [213]:
# WITH PIPELINES

In [214]:
ct = ColumnTransformer(transformers=[
    ('ct1',SimpleImputer(),['fever']),
    ('ct2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('ct3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
],remainder='passthrough')

In [215]:
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('ct1', SimpleImputer(), ['fever']),
                                ('ct2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('ct3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [216]:
ct.fit(x_train)

ct.transform(x_train)
ct.transform(x_test)



array([[104.,   0.,   0.,   0.,   1.,   0.,  17.],
       [ 98.,   0.,   1.,   1.,   0.,   0.,  83.],
       [101.,   1.,   0.,   1.,   0.,   0.,  68.],
       [ 99.,   0.,   1.,   0.,   0.,   0.,  72.],
       [102.,   1.,   1.,   1.,   0.,   0.,  20.],
       [103.,   0.,   0.,   0.,   1.,   0.,  50.],
       [ 98.,   1.,   0.,   0.,   1.,   0.,  71.],
       [ 99.,   0.,   0.,   0.,   0.,   1.,  14.],
       [101.,   0.,   0.,   1.,   0.,   0.,  75.],
       [103.,   0.,   1.,   0.,   1.,   0.,  60.],
       [ 98.,   0.,   0.,   0.,   0.,   0.,  64.],
       [101.,   0.,   1.,   1.,   0.,   0.,  15.],
       [ 98.,   1.,   1.,   0.,   1.,   0.,  34.],
       [ 98.,   0.,   0.,   0.,   1.,   0.,  26.],
       [ 99.,   1.,   0.,   1.,   0.,   0.,  59.],
       [101.,   0.,   0.,   0.,   0.,   1.,  65.],
       [100.,   0.,   1.,   0.,   0.,   0.,  80.],
       [101.,   0.,   0.,   0.,   1.,   0.,   8.],
       [ 99.,   1.,   0.,   0.,   1.,   0.,  25.],
       [103.,   0.,   1.,   0.,